In [4]:
import json
import pandas as pd
from pandas import json_normalize

In [5]:
json_path = "C:/Users/hamid/OneDrive/Desktop/masoume/springboard_data_engineering/capstone project/EDA/drug-event-0001-of-0005.json"

# Load raw nested JSON lines
records = []

with open(json_path, "r") as f:
    data = json.load(f)

# Step 2: Extract the list of reports from the "results" key
records = data["results"]

###  Main Report Table
df_main = pd.json_normalize(
    data=records,
    sep="."
)

print(df_main.head())

  safetyreportid transmissiondateformat transmissiondate serious  \
0      7988601-5                    102         20120403       1   
1      7533262-2                    102         20111010       1   
2      7544040-2                    102         20111010       1   
3      5567969-3                    102         20080405       1   
4      5562608-X                    102         20080405       1   

  seriousnessother receivedateformat receivedate receiptdateformat  \
0                1               102    20111218               102   
1              NaN               102    20110608               102   
2                1               102    20110613               102   
3                1               102    20071220               102   
4                1               102    20071217               102   

  receiptdate fulfillexpeditecriteria  ... receiver.receiverorganization  \
0    20050926                       1  ...                           NaN   
1    20050919     

In [6]:
print('##########################Check Homogeneity report columns #########################')
for column in df_main.columns:
    non_null_series = df_main[column].dropna()
    
    if non_null_series.empty:
        print(f"⚠️ Column: {column} has only null values — can't assess homogeneity.\n")
        continue
    
    types = non_null_series.map(type).value_counts()
    
    if len(types) > 1:
        print(f"❌ Column: {column} has mixed types:")
        print(types, "\n")
    else:
        print(f"✅ Column: {column} is homogeneous: {types.index[0].__name__}\n")

##########################Check Homogeneity report columns #########################
✅ Column: safetyreportid is homogeneous: str

✅ Column: transmissiondateformat is homogeneous: str

✅ Column: transmissiondate is homogeneous: str

✅ Column: serious is homogeneous: str

✅ Column: seriousnessother is homogeneous: str

✅ Column: receivedateformat is homogeneous: str

✅ Column: receivedate is homogeneous: str

✅ Column: receiptdateformat is homogeneous: str

✅ Column: receiptdate is homogeneous: str

✅ Column: fulfillexpeditecriteria is homogeneous: str

✅ Column: companynumb is homogeneous: str

⚠️ Column: receiver has only null values — can't assess homogeneity.

✅ Column: primarysource.reportercountry is homogeneous: str

✅ Column: primarysource.qualification is homogeneous: str

✅ Column: sender.senderorganization is homogeneous: str

✅ Column: patient.patientonsetage is homogeneous: str

✅ Column: patient.patientonsetageunit is homogeneous: str

✅ Column: patient.patientsex is homog

In [17]:
print('#########################Missing Value percentage############################')
percent_missing = df_main.isnull().sum() * 100 / len(df_main)

missing_value_df = pd.DataFrame({
    'column_name': df_main.columns,
    'percent_missing': percent_missing.values
})

# Sort by percentage
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)

# ✅ Print result
print(missing_value_df)


#########################Missing Value percentage############################
                                    column_name  percent_missing
40                                primarysource       100.000000
11                                     receiver       100.000000
36        patient.patientdeath.patientdeathdate       100.000000
35  patient.patientdeath.patientdeathdateformat       100.000000
24                                 occurcountry        99.991667
25                                   reporttype        99.991667
26                                    duplicate        99.991667
27              reportduplicate.duplicatesource        99.991667
28                reportduplicate.duplicatenumb        99.991667
29                            sender.sendertype        99.991667
30                        receiver.receivertype        99.991667
31                receiver.receiverorganization        99.991667
22                          safetyreportversion        99.991667
32          

In [8]:

### ---------- STEP 2: Flatten patient.drug ----------
df_drug = pd.json_normalize(
    data=records,
    record_path=["patient", "drug"],
    meta=["safetyreportid"],    
    record_prefix="drug.",
    meta_prefix="report.",
    sep="."
)

print(df_drug.head())


  drug.drugcharacterization drug.medicinalproduct drug.drugauthorizationnumb  \
0                         1             VALSARTAN                     021283   
1                         1             CLOZAPINE                     019758   
2                         1             CLOZAPINE                     019758   
3                         1          ARIPIPRAZOLE                        NaN   
4                         1           HALOPERIDOL                     015923   

                     drug.openfda.application_number  \
0  [ANDA202223, ANDA204038, ANDA209261, ANDA20534...   
1  [ANDA206433, ANDA090308, NDA203479, ANDA202873...   
2  [ANDA206433, ANDA090308, NDA203479, ANDA202873...   
3  [ANDA207240, ANDA205363, ANDA205064, ANDA21635...   
4  [ANDA216918, ANDA070278, ANDA071173, ANDA21878...   

                             drug.openfda.brand_name  \
0                                [VALSARTAN, DIOVAN]   
1                   [CLOZAPINE, VERSACLOZ, CLOZARIL]   
2             

In [9]:
print('##########################Check Homogeneity of drug columns #########################')

for column in df_drug.columns:
    non_null_series = df_drug[column].dropna()
    
    if non_null_series.empty:
        print(f"⚠️ Column: {column} has only null values — can't assess homogeneity.\n")
        continue
    
    types = non_null_series.map(type).value_counts()
    
    if len(types) > 1:
        print(f"❌ Column: {column} has mixed types:")
        print(types, "\n")
    else:
        print(f"✅ Column: {column} is homogeneous: {types.index[0].__name__}\n")


##########################Check Homogeneity of drug columns #########################
✅ Column: drug.drugcharacterization is homogeneous: str

✅ Column: drug.medicinalproduct is homogeneous: str

✅ Column: drug.drugauthorizationnumb is homogeneous: str

✅ Column: drug.openfda.application_number is homogeneous: list

✅ Column: drug.openfda.brand_name is homogeneous: list

✅ Column: drug.openfda.generic_name is homogeneous: list

✅ Column: drug.openfda.manufacturer_name is homogeneous: list

✅ Column: drug.openfda.product_ndc is homogeneous: list

✅ Column: drug.openfda.product_type is homogeneous: list

✅ Column: drug.openfda.route is homogeneous: list

✅ Column: drug.openfda.substance_name is homogeneous: list

✅ Column: drug.openfda.rxcui is homogeneous: list

✅ Column: drug.openfda.spl_id is homogeneous: list

✅ Column: drug.openfda.spl_set_id is homogeneous: list

✅ Column: drug.openfda.package_ndc is homogeneous: list

✅ Column: drug.openfda.nui is homogeneous: list

✅ Column: drug

In [16]:
print('#########################Missing Value percentage############################')
percent_missing = df_drug.isnull().sum() * 100 / len(df_drug)

missing_value_df = pd.DataFrame({
    'column_name': df_drug.columns,
    'percent_missing': percent_missing.values
})

# Sort by percentage
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)

# ✅ Print result
print(missing_value_df)

#########################Missing Value percentage############################
                                 column_name  percent_missing
33              drug.drugstructuredosageunit        99.998137
32              drug.drugstructuredosagenumb        99.998137
31  drug.activesubstance.activesubstancename        99.964604
30                           drug.actiondrug        99.964604
29                       drug.drugdosageform        99.964604
35                        drug.drugbatchnumb        95.240136
27            drug.drugtreatmentdurationunit        93.697604
26                drug.drugtreatmentduration        93.533664
28               drug.openfda.pharm_class_pe        91.296248
34               drug.openfda.pharm_class_cs        82.925966
25                          drug.drugenddate        80.975446
24                    drug.drugenddateformat        80.975446
2                 drug.drugauthorizationnumb        79.606170
16              drug.openfda.pharm_class_moa        78

In [13]:

### ---------- STEP 2: Flatten patient.reaction ----------
df_reaction = pd.json_normalize(
    data=records,
    record_path=["patient", "reaction"],
    meta=["safetyreportid"],    
    record_prefix="reaction.",
    meta_prefix="report.",
    sep="."
)

print(df_reaction.head())


          reaction.reactionmeddrapt reaction.reactionmeddraversionpt  \
0               UPPER LIMB FRACTURE                              NaN   
1                   SPEECH DISORDER                              NaN   
2  WHITE BLOOD CELL COUNT INCREASED                              NaN   
3                         DYSPHAGIA                              NaN   
4        NEUTROPHIL COUNT INCREASED                              NaN   

  reaction.reactionoutcome report.safetyreportid  
0                      NaN             7988601-5  
1                      NaN             7533262-2  
2                      NaN             7533262-2  
3                      NaN             7533262-2  
4                      NaN             7533262-2  


In [14]:
print('##########################Check Homogeneity of reaction columns #########################')

for column in df_reaction.columns:
    non_null_series = df_reaction[column].dropna()
    
    if non_null_series.empty:
        print(f"⚠️ Column: {column} has only null values — can't assess homogeneity.\n")
        continue
    
    types = non_null_series.map(type).value_counts()
    
    if len(types) > 1:
        print(f"❌ Column: {column} has mixed types:")
        print(types, "\n")
    else:
        print(f"✅ Column: {column} is homogeneous: {types.index[0].__name__}\n")

##########################Check Homogeneity of reaction columns #########################
✅ Column: reaction.reactionmeddrapt is homogeneous: str

✅ Column: reaction.reactionmeddraversionpt is homogeneous: str

✅ Column: reaction.reactionoutcome is homogeneous: str

✅ Column: report.safetyreportid is homogeneous: str



In [18]:
print('#########################Missing Value percentage############################')
percent_missing = df_reaction.isnull().sum() * 100 / len(df_reaction)

missing_value_df = pd.DataFrame({
    'column_name': df_reaction.columns,
    'percent_missing': percent_missing.values
})

# Sort by percentage
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)


# ✅ Print result
print(missing_value_df)

#########################Missing Value percentage############################
                        column_name  percent_missing
1  reaction.reactionmeddraversionpt        99.998067
2          reaction.reactionoutcome        99.998067
0         reaction.reactionmeddrapt         0.000000
3             report.safetyreportid         0.000000


ModuleNotFoundError: No module named 'graphviz'

![My Diagram](ER.png)